# Build your own Chatbot:

##### Making use of NLTK in Python to create a virtual tour guide for Jaipur!

## Step 1. Import required libraries:

In [1]:
import io
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer

## Step 2. Data Reading:

In [3]:
#reading the text file from which response will be fetched

with open('jaipur.txt','r', encoding='utf8', errors ='ignore') as file: 
    raw = file.read().lower() #converting the contents to the file into lowercase
    

nltk.download('wordnet') # first-time use only

sent_tokens = nltk.sent_tokenize(raw) # converts to list of sentences 

word_tokens = nltk.word_tokenize(raw) # converts to list of words

[nltk_data] Downloading package wordnet to C:\Users\ANUSHA
[nltk_data]     AGRAWAL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Print a simple example to see sentence and word tokens: 

In [4]:
sent_tokens[:5]

['jaipur is in the state of rajasthan, towards the north-western part of india.',
 'rajasthan shares proximity withgujarat, madhya pradesh, uttar pradesh, punjab, and haryana.',
 'the best time to visit jaipur is between the months of october to march, with occasional showers bathing the city’s streets accompanied with snug weather.',
 'the fastest way to reach jaipur is via the indira gandhi international airport in delhi with an airtime of an hour.',
 'once you reach the jaipur airport at sanganer, you could hire a taxi or cab to your desired destination.']

In [5]:
word_tokens[:5]

['jaipur', 'is', 'in', 'the', 'state']

## Step 3: Preprocessing and Handling Raw Text:

In [6]:
lemmer = WordNetLemmatizer()

#user defined function for lemmatization
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

#removing punctuation marks from the text file to clean it
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

#user defined fucntion 
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

## Step 4: Programming a Greet Response: 

In [7]:
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey","hii"] #making a list of user greeting inputs
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"] #making a list of the robo outputs

#user defined function to greet the user
def greeting(sentence):
    for word in sentence.split(): 
        if word.lower() in GREETING_INPUTS: 
            return random.choice(GREETING_RESPONSES)

## Step 5: Writing the Response Function:

In [8]:
#user defined function to fetch bot response

def response(user_response):
    robo_response='' #empty string because the response is not known initially
    sent_tokens.append(user_response) #appending user response to sentence tokens
    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') #making the tfidf model to assign weights
    tfidf = TfidfVec.fit_transform(sent_tokens) #fitting the model on sent tokens
    
    vals = cosine_similarity(tfidf[-1], tfidf) #finding similarity between user response and all other sentences
    idx=vals.argsort()[0][-2] #sorting the array in accordance with the index number
    
    flat = vals.flatten() #converting 2-D array into 1-D array
    flat.sort() #sorting the array in accordance with values
    
    req_tfidf = flat[-2] #reaching out for the second last sentence
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you" #if the response is not there in the text file, req_tfidf will be 0
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx] #returing the relevant response
        return robo_response

In [9]:
#CHATBOT Coding
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!") #welcome statement by the chatbot

while(flag==True): #chatbot will be open to questions for as long as the flag is true
    user_response = input() #taking input of question
    user_response=user_response.lower() #converting the question to lowercase
    
    if(user_response!='bye'): #if the user input is not equals to bye
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..") #bot says welcome if the user writes thanks
        else:
            greet = greeting(user_response) 
            if(greet!=None):
                print("ROBO: "+greet) #bot will greet the user by calling greeting function
            else:
                reply = response(user_response)  #relevant response will be fetched by calling the response function
                print("ROBO: ",end="")
                print(reply)
                sent_tokens.remove(user_response) #removing the user input from sent tokens
    else:
        flag=False
        print("ROBO: Bye! take care..")  #robo says bye
    print()

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hey
ROBO: hi there

bye
ROBO: Bye! take care..



# Code with Integration of Text to Speech Converter

In [3]:
!pip install speech_recognition

  ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition


In [9]:
import speech_recognition as sr
from gtts import gTTS

ModuleNotFoundError: No module named 'speech_recognition'

In [12]:
###WITH SPEECH TO TEXT AND  TEXT TO SPEECH
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")

while(flag==True):
    r = sr.Recognizer()
    try:
        with sr.Microphone() as source:
            print("Say Something!")
            audio = r.listen(source)
        user_response = r.recognize_google(audio, language = 'en-US') #Understanding the audio in order to print it (the audio is supposed to be in english only)
        
        print(user_response) #priting the input given by user
        
        user_response=user_response.lower() #connverting user input to lower case
        
        if(user_response!='bye'): #if the user input is not equals to bye
            
            if(user_response=='thanks' or user_response=='thank you' ): 
                flag=False
                thank = " You are welcome.." #assigning value to variable thank
                tts = gTTS(thank) #converting the value of variable thank into audio
                tts.save("thank.mp3") #saving the audio file
                os.system("thank.mp3") #output in the form of speech
                print("ROBO:"+thank) #output in the form of text
            
            else:
                greet = greeting(user_response) #calling greeting function
                if(greet!=None):
                    tts = gTTS(greet) #converting the value of variable greet into audio
                    tts.save("greet.mp3") #saving the audio file
                    os.system("greet.mp3") #output in the form of speech
                    print("ROBO:"+greet) #output in the form of text
                else:
                    reply = response(user_response)
                    tts = gTTS(reply) #converting the value of variable reply into audio
                    tts.save("reply.mp3") #saving the audio file
                    os.system("reply.mp3") #output in the form of speech
                    print("ROBO: "+reply) #output in the form of text
                    sent_tokens.remove(user_response) #removing the user input from sent tokens
        else:
            flag=False
            bye = " Bye! take care.."  #assigning value to variable bye
            tts = gTTS(bye) #converting the value of variable bye into audio
            tts.save("bye.mp3") #saving the audio file
            os.system("bye.mp3") #output in the form of text
            print("ROBO:"+bye)  #output in the form of text
        print()
    except:
        pass

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!


NameError: name 'sr' is not defined